Reuben Feinman

Math Tools for Neural and Cog. Sci.

Homework #4

11/2/17

# Problem #1

## (a)

Let $B$ denote the case that Mom's second gene is the dominant brown-eyed gene, and $b$ the case that it is the recessive blue-eyed gene. We are given the prior probabilities $P(B) = P(b) = 50\%$. Given our observation, $O$, that the child has brown eyes, we can define the conditional probabilities of this observation for each case of Mom as follows (using understanding of genetics):

$P(O|B) = 1$

$P(O|b) = \frac{1}{2}$

Now, what we ultimately want to compute is $P(O|b)$. To do this, we use Baye's Rule:

$P(b|O) = \frac{P(O|b)*P(b)}{P(O)} = \frac{P(O|b)*P(b)}{P(O|b)*P(b) + P(O|B)*P(B)} 
= \frac{\frac{1}{2}*\frac{1}{2}}{\frac{1}{2}*\frac{1}{2} + 1*\frac{1}{2}} 
= \frac{\frac{1}{4}}{\frac{3}{4}} = \frac{1}{3}$

So the probability is $\frac{1}{3}$.

## (b)

Now we have two observations, $O_1, O_2$. Assuming that these two events are conditionally independent (this would be the case in biology), we can compute our joint conditional probabilities as follows:

$P(O_1, O_2|B) = P(O_1|B)*P(O_2|B) = 1*1 = 1$

$P(O_1, O_2|b) = P(O_1|b)*P(O_2|b) = \frac{1}{2}*\frac{1}{2} = \frac{1}{4}$

We can use Baye's Rule to compute $P(b|O_1,O_2)$:

$P(b|O_1,O_2) = \frac{P(O_1,O_2|b)*P(b)}{P(O_1,O_2)}$.

We can compute the denominator $P(O_1,O_2) = P(O_1)*P(O_2)$ because the events are independent. Recall from part (a) that $P(O) = \frac{3}{4}$ and thus $P(O_1,O_2) = \frac{3}{4}*\frac{3}{4} = \frac{9}{16}$. So we have

$P(b|O_1,O_2) = \frac{P(O_1,O_2|b)*P(b)}{P(O_1,O_2)} = \frac{\frac{1}{4}*\frac{1}{2}}{\frac{9}{16}}
= \frac{\frac{1}{8}}{\frac{9}{16}} = \frac{2}{9}$.

So the probability is $\frac{2}{9}$.

## (c)

Extending upon the generalization we made to get from (a) to (b) above, we get an equation for our target probability as follows:
$P(b|O_1, ..., O_N) = \frac{P(O_1, ..., O_N|b)*P(b)}{P(O_1, ..., O_N)} = \frac{P(O|b)^N*P(b)}{P(O)^N}
= \frac{(\frac{1}{2})^N*\frac{1}{2}}{(\frac{3}{4})^N} = \frac{2^{N-1}}{3^N}$.

So the probability is $\frac{2^{N-1}}{3^N}$.

# Problem #2